In [1]:
import preprocessing as pp
import sto_mod as sm
import pandas as pd
import numpy as np
import time 
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

In [5]:
# Read and Preprocess the data
(cus_df, cand_df, sup_df, vehicle_df, distance_w_to_s_df, _, _,
    demand_cus_period_scene_df) = pp.read_and_prep_data()

time_limit_s = 3600
n_cus_clusters = 50
n_scene_clusters_list = [5,10]

print(f'Now running Case N_cus_cluster = {n_cus_clusters}')

# Customer Clustering
cus_df, clust_center_df = pp.const_cluster_by_cus_loc(cus_df, n_clusters=n_cus_clusters, size_min=1, size_max=20,
                                                        random_state=42)
agg_dem_cus_period_scene_df = pp.agg_dem_cus_period_scene(demand_cus_period_scene_df, cus_df)
distance_w_to_cluster_df = pp.create_dis_mat_df(cand_df, clust_center_df, 'cityblock')

# Create Cost
tra_cost_w_to_cluster = pp.calculate_cost_from_w_to_cluster(distance_w_to_cluster_df, vehicle_df)
tra_cost_w_to_s = pp.calculate_cost_from_w_to_s(distance_w_to_s_df, vehicle_df, sup_df)

# time_obj_gap_dict = {}

for n_scene_clusters in n_scene_clusters_list:
    print(f'Now running Case N_scene_cluster = {n_scene_clusters}')
    # Scenarios Clustering
    _, scene_cluster_center_df = pp.constrained_kmeans_clustering(agg_dem_cus_period_scene_df,
                                                                    n_clusters=n_scene_clusters,
                                                                    size_min=np.floor(
                                                                        len(agg_dem_cus_period_scene_df.columns) /
                                                                        n_scene_clusters),
                                                                    size_max=len(agg_dem_cus_period_scene_df.columns),
                                                                    random_state=42)
    agg_dem_cus_period_clus_scene_df = pp.agg_scene_df(agg_dem_cus_period_scene_df, scene_cluster_center_df)

    solve_time, objective_value, mip_gap_percentage, x_sol, y_sol, o_sol, z_sol = sm.sto_model(agg_dem_cus_period_clus_scene_df, cand_df, sup_df, clust_center_df, tra_cost_w_to_s,
                                                                                            tra_cost_w_to_cluster, time_limit_s)

    time_obj_gap_dict[n_scene_clusters] = [solve_time, objective_value, mip_gap_percentage]

Now running Case N_cus_cluster = 50
Now running Case N_scene_cluster = 5


/Users/thee/miniconda/envs/trm_conda/lib/python3.11/site-packages/k_means_constrained/k_means_constrained_.py:464: RuntimeWarning: invalid value encountered in cast
  costs = np.around(costs * 1000, 0).astype('int32')  # Times by 1000 to give extra precision


FICO Xpress v9.4.2, Hyper, solve started 1:07:23, Feb 14, 2025
Heap usage: 971MB (peak 971MB, 289MB system)
Minimizing MILP MEWLP_Stochastics using up to 11 threads and up to 18GB memory, with these control settings:
MAXTIME = 3600
OUTPUTLOG = 1
MIPRELSTOP = .001
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
   1221390 rows      1338040 cols     10237920 elements   1104840 entities
Presolved problem has:
   1198737 rows      1319347 cols     10081336 elements   1086147 entities
LP relaxation tightened
Presolve finished in 42 seconds
Heap usage: 1437MB (peak 2027MB, 289MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 1.22e-02,  1.33e+02]
  RHS and bounds [min,max] : [ 1.00e+00,  1.53e+07] / [ 1.00e+00,  9.36e+02]
  Objective      [min,max] : [ 2.45e-04,  6.25e+06] / [ 5.01e-01,  6.88e+06]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch and 

/Users/thee/miniconda/envs/trm_conda/lib/python3.11/site-packages/k_means_constrained/k_means_constrained_.py:464: RuntimeWarning: invalid value encountered in cast
  costs = np.around(costs * 1000, 0).astype('int32')  # Times by 1000 to give extra precision


FICO Xpress v9.4.2, Hyper, solve started 2:22:27, Feb 14, 2025
Heap usage: 1938MB (peak 1938MB, 575MB system)
Minimizing MILP MEWLP_Stochastics using up to 11 threads and up to 18GB memory, with these control settings:
MAXTIME = 3600
OUTPUTLOG = 1
MIPRELSTOP = .001
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
   2433890 rows      2438040 cols     20225920 elements   2204840 entities
Presolved problem has:
   2392119 rows      2400229 cols     19914624 elements   2167029 entities
LP relaxation tightened
Presolve finished in 68 seconds
Heap usage: 2822MB (peak 4045MB, 575MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 1.21e-02,  1.34e+02]
  RHS and bounds [min,max] : [ 1.00e+00,  1.53e+07] / [ 1.00e+00,  5.84e+02]
  Objective      [min,max] : [ 2.45e-04,  6.25e+06] / [ 5.01e-01,  6.88e+06]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch an

In [ ]:
adfasd

In [6]:
time_obj_gap_dict

{3: [1182.6707799434662, 40677339.88299683, 0.09807651626876435],
 5: [3600.366682291031, 41461046.694164544, 2.172641144570973],
 10: [3601.3514869213104, 50211482.286392644, 18.101819089096708]}

In [ ]:
asdfasdf

In [13]:
40677340 - 39842948

834392

In [12]:
100*(40677340 - 39842948)/39842948

2.0942024671467583

In [7]:
test_dict = {   3: [1182.6707799434662, 40677339.88299683, 0.09807651626876435],
                5: [3600.366682291031, 41461046.694164544, 2.172641144570973],
                10: [3601.3514869213104, 50211482.286392644, 18.101819089096708],
                15: [3600, 111273302.64218375, 100]}

In [8]:
test_dict

{3: [1182.6707799434662, 40677339.88299683, 0.09807651626876435],
 5: [3600.366682291031, 41461046.694164544, 2.172641144570973],
 10: [3601.3514869213104, 50211482.286392644, 18.101819089096708],
 15: [3600, 111273302.64218375, 100]}

# Appendix

In [ ]:
# Read and Preprocess the data
customer_df, candidate_df, supplier_df, vehicle_df, distance_w_to_s_df, distance_w_to_c_df, demand_cus_period_df, demand_cus_period_scene_df = pp.read_and_prep_data()

# Customer Clustering
customer_df, cluster_center_df = pp.const_cluster_by_cus_loc(customer_df, n_clusters=50, size_min=1, size_max=20, random_state=42)

agg_dem_cus_period_scene_df = pp.agg_dem_cus_period_scene(demand_cus_period_scene_df, customer_df)

distance_w_to_cluster_df = pp.create_dis_mat_df(candidate_df, cluster_center_df,'cityblock')

# Create Cost
cost_w_to_cluster = pp.calculate_cost_from_w_to_cluster(distance_w_to_cluster_df, vehicle_df)
cost_w_to_s = pp.calculate_cost_from_w_to_s(distance_w_to_s_df, vehicle_df, supplier_df)


In [ ]:
# n_clus_scene = 3
time_limit_s = 3600
time_obj_gap_dict = {}
n_clusters_scene_list = [4,5,10]

for each_n_clusters_scene in n_clusters_scene_list:

    df,scene_cluster_center_df = pp.constrained_kmeans_clustering(agg_dem_cus_period_scene_df, n_clusters=each_n_clusters_scene, 
                                                                    size_min=np.floor(len(agg_dem_cus_period_scene_df.columns)/n_each_n_clusters_sceneclus_scene), 
                                                                    size_max=len(agg_dem_cus_period_scene_df.columns), random_state=42)
    agg_dem_cus_period_clus_scene_df = pp.agg_scene_df(agg_dem_cus_period_scene_df, scene_cluster_center_df)

    solve_time, obj_value, mip_gap_percent, setup_cost, operating_cost, tra_w_s_cost, recourse = sm.sto_model(  agg_dem_cus_period_clus_scene_df,
                                                                                                                candidate_df,
                                                                                                                supplier_df,
                                                                                                                cluster_center_df,
                                                                                                                cost_w_to_s,
                                                                                                                cost_w_to_cluster,
                                                                                                                time_limit_s)

    time_obj_gap_dict[each_n_clusters_scene] = [solve_time, obj_value, mip_gap_percent, setup_cost, operating_cost, tra_w_s_cost, recourse]

/Users/thee/miniconda/envs/trm_conda/lib/python3.11/site-packages/k_means_constrained/k_means_constrained_.py:464: RuntimeWarning: invalid value encountered in cast
  costs = np.around(costs * 1000, 0).astype('int32')  # Times by 1000 to give extra precision


FICO Xpress v9.3.5, Hyper, solve started 19:14:05, Feb 13, 2025
Heap usage: 1864MB (peak 1864MB, 620MB system)
Minimizing MILP MEWLP_Stochastics using up to 11 threads and up to 18GB memory, with these control settings:
MAXTIME = 3600
OUTPUTLOG = 1
MIPRELSTOP = .001
Original problem has:
   3376390 rows      3538040 cols     13502720 elements    664840 entities
Presolved problem has:
    722080 rows       886739 cols      6147605 elements    653539 entities
LP relaxation tightened
Presolve finished in 36 seconds
Heap usage: 2618MB (peak 4498MB, 620MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 9.88e-14,  1.32e+02]
  RHS and bounds [min,max] : [ 1.00e+00,  1.53e+07] / [ 1.00e+00,  9.36e+02]
  Objective      [min,max] : [ 1.61e-04,  6.25e+06] / [ 5.01e-01,  6.88e+06]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch and bound tree memory usage below 9.5GB
Starting concurrent s

In [ ]:
agg_dem_cus_period_clus_scene_df

0              1              2
Cluster ProductIndex PeriodIndex                                             
0       0            0              4708.121212    4584.176471    4684.272727
                     1              9466.242424    9428.000000    9825.454545
                     2             13200.909091   13249.352941   13835.060606
                     3             18252.969697   17999.382353   19496.030303
                     4             22165.484848   21969.794118   24170.757576
...                                         ...            ...            ...
49      3            5            133272.696970  134476.794118  150438.454545
                     6            155566.878788  153380.058824  175321.393939
                     7            179462.696970  173250.617647  205351.878788
                     8            196845.666667  192672.558824  232152.727273
                     9            217813.000000  214743.735294  262695.272727

[2000 rows x 3 columns]

In [ ]:
# time_limit_s = 3600

# model = xp.problem(name= 'MEWLP_Stochastics')
# # Sets and Notation
# S = list(supplier_df.index) # Supplier Index
# S_P0 = list(supplier_df[supplier_df['SupplierProductGroup'] == 0]['SupplierProductGroup'].index)
# S_P1 = list(supplier_df[supplier_df['SupplierProductGroup'] == 1]['SupplierProductGroup'].index)
# S_P2 = list(supplier_df[supplier_df['SupplierProductGroup'] == 2]['SupplierProductGroup'].index)
# S_P3 = list(supplier_df[supplier_df['SupplierProductGroup'] == 3]['SupplierProductGroup'].index)
# S_P_dict = {0:S_P0, 1:S_P1, 2:S_P2, 3:S_P3}

# W = list(candidate_df.index) # Warehouse Index
# C = list(cluster_center_df.index) # Cluster Index
# P = list(agg_dem_cus_period_clus_scene_df.reset_index()['ProductIndex'].unique())
# T = list(agg_dem_cus_period_clus_scene_df.reset_index()['PeriodIndex'].unique())
# Phi = list(agg_dem_cus_period_clus_scene_df.columns)

# # Output Variables
# x = np.array([xp.var(f'x_{w}_{c}_{t}_{xi}', vartype = xp.binary) for w in W for c in C for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(C), len(T), len(Phi))
# y = np.array([xp.var(f'y_{w}_{t}', vartype = xp.binary) for w in W for t in T], dtype = xp.npvar).reshape(len(W), len(T))
# o = np.array([xp.var(f'o_{w}', vartype = xp.binary) for w in W], dtype = xp.npvar).reshape(len(W))
# v = np.array([xp.var(f'v_{w}_{c}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T), len(Phi))
# z = np.array([xp.var(f'z_{w}_{s}_{t}', vartype = xp.continuous, lb = 0) for w in W for s in S for t in T], dtype = xp.npvar).reshape(len(W), len(S), len(T))
# # a = np.array([xp.var(f'a_{w}_{c}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W),len(C), len(P), len(T), len(Phi))
# # b = np.array([xp.var(f'b_{w}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for w in W for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(P), len(T), len(Phi))

# # model.addVariable(x, y, o, v, z, a, b)
# model.addVariable(x, y, o, v, z)


# # Constraints 
# for w in W:
#     for t in T:
#         model.addConstraint(xp.constraint(o[w] >= y[w,t]))
#         if t != 0:
#             model.addConstraint(xp.constraint(y[w, t] >= y[w,t-1]))    

#         for c in C:
#             for xi in Phi:
#                 model.addConstraint(xp.constraint(x[w,c,t,xi] <= y[w,t]))  

# for xi in Phi:
#     for t in T:
#         for w in W:
#             Capacity_W = candidate_df.loc[w,'Capacity']
#             model.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t,xi] for c in C for p in P) <= Capacity_W * y[w,t]))
#         for c in C:
#             model.addConstraint(xp.constraint(xp.Sum(x[w,c,t,xi] for w in W) == 1))

# for xi in Phi:
#     for t in T:
#         for c in C:
#             for p in P:
#                 Demand = agg_dem_cus_period_scene_df.loc[(c,p,t),xi]
#                 for w in W:
#                     # model.addConstraint(xp.constraint(v[w,c,p,t,xi] + a[w,c,p,t,xi] == Demand * x[w,c,t,xi]))
#                     model.addConstraint(xp.constraint(v[w,c,p,t,xi] >= Demand * x[w,c,t,xi]))

# for xi in Phi:
#     for t in T:
#         for w in W:
#             for p in P:
#                 # model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) + b[w,p,t,xi] >= xp.Sum(v[w,c,p,t,xi] for c in C)))
#                 model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) >= xp.Sum(v[w,c,p,t,xi] for c in C)))
                
# for s in S:
#     Capacity_S = supplier_df.loc[s,'SupplierCapacity']
#     for t in T:
#         model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for w in W) <= Capacity_S))

# Setup_cost = xp.Sum(candidate_df.loc[w,'Setup'] * o[w] for w in W)
# Operating_cost = xp.Sum(candidate_df.loc[w,'Operating'] * y[w,t] for w in W for t in T)
# Tra_w_s_cost = xp.Sum(cost_w_to_s.loc[w,s] * z[w,s,t] for w in W for s in S for t in T)

# equal_prob = 1/len(agg_dem_cus_period_clus_scene_df.columns)
# # ld = 10000000000
# # gm = 10000000000/2
# Recourse = 0

# for xi in Phi:
#     Tra_w_c_cost = xp.Sum(cost_w_to_cluster.loc[w,c] * v[w,c,p,t,xi] for w in W for c in C for p in P for t in T)
#     # Pen_w_c = ld* xp.Sum(a[w,c,p,t,xi] for w in W for c in C for p in P for t in T)
#     # Pen_w_s = gm* xp.Sum(b[w,p,t,xi] for w in W for p in P for t in T)
#     # Recourse += equal_prob * (Tra_w_c_cost + Pen_w_c + Pen_w_s)
#     Recourse += equal_prob * (Tra_w_c_cost)

# obj = Setup_cost + Operating_cost + Tra_w_s_cost + Recourse
# # model.addObjective(Setup_cost, Operating_cost, Tra_w_s_cost, Recourse)
# model.setObjective(obj, sense = xp.minimize)

# model.setControl('miprelstop', 1e-3)
# model.setControl('maxtime', time_limit_s)
# tic_time = time.time()
# # Solve the problem
# model.solve()
# toc_time = time.time()
# solve_time = toc_time - tic_time
# obj_value = model.getObjVal()

# mip_gap_percent = 100*(obj_value - model.getAttrib('bestbound'))/obj_value
# print(f'Solving Time: {solve_time}')
# print(f'Objective Value: {obj_value}')
# print(f'%Gaps: {mip_gap_percent}')


In [ ]:
obj_value

40677339.88299679

In [ ]:
time_obj_gap_dict = {}
time_obj_gap_dict[n_clus_scene] = [solve_time, obj_value, mip_gap_percent]

In [ ]:
asdfasdfasdfasdf

NameError: name 'asdfasdfasdfasdf' is not defined

In [ ]:
time_obj_gap_dict

{3: [1022.6493480205536, 40677339.88299679, 0.07253205132098663]}

50 Clusters

{3: [1022.6493480205536, 40677339.88299679, 0.07253205132098663]}

100 Clusters (Optional)